<a href="https://colab.research.google.com/github/vijaygwu/SEAS8525/blob/main/Class_6_Nemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install nemo_toolkit[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 53.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.2/150.2 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 99.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━

In [ ]:
import nemo
import nemo.collections.asr as nemo_asr
import nemo.collections.nlp as nemo_nlp
import nemo.collections.tts as nemo_tts

In [ ]:

# Download pre-trained QuartzNet model
model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")




[NeMo I 2024-02-19 01:58:14 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.22.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.
[NeMo I 2024-02-19 01:58:14 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.22.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo
[NeMo I 2024-02-19 01:58:14 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2024-02-19 01:58:14 features:289] PADDING: 16
[NeMo I 2024-02-19 01:58:16 save_restore_connector:249] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.22.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['we the people of the united states in order to form a more perfect union establish justice end sure domestic tranquillity provide for the common defense promote the general welfare and secure the blessings of liberty to ourselves in our posterity to ordain and establish this constitution for the united states of america']


In [ ]:
!wget https://www2.cs.uic.edu/~i101/SoundFiles/preamble.wav

--2024-02-19 02:00:16--  https://www2.cs.uic.edu/~i101/SoundFiles/preamble.wav
Resolving www2.cs.uic.edu (www2.cs.uic.edu)... 131.193.32.16
Connecting to www2.cs.uic.edu (www2.cs.uic.edu)|131.193.32.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 842264 (823K) [audio/x-wav]
Saving to: ‘preamble.wav’

preamble.wav        100%[===================>] 822.52K  --.-KB/s    in 0.1s    

2024-02-19 02:00:16 (5.90 MB/s) - ‘preamble.wav’ saved [842264/842264]



In [ ]:
# Transcribe audio
transcript = model.transcribe(["/content/preamble.wav"])

print(transcript)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['we the people of the united states in order to form a more perfect union establish justice end sure domestic tranquillity provide for the common defense promote the general welfare and secure the blessings of liberty to ourselves in our posterity to ordain and establish this constitution for the united states of america']
